In [ ]:
# =============================================================================
# CELL 1: Environment Setup & Path Configuration
# =============================================================================

import os
import sys
from pathlib import Path

# Detect environment
IN_COLAB = 'google.colab' in sys.modules
IN_KAGGLE = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ

if IN_COLAB:
    # Clone repo if in Colab
    REPO_URL = 'https://github.com/aayushakumar/ood-eval-toxic-classifiers.git'
    REPO_DIR = '/content/ood-eval-toxic-classifiers'
    
    if not os.path.exists(REPO_DIR):
        print(f"Cloning repository...")
        !git clone {REPO_URL} {REPO_DIR}
    
    os.chdir(REPO_DIR)
    ROOT_DIR = Path(REPO_DIR)
    
    # Install requirements
    !pip install -q transformers datasets scikit-learn tqdm
    
elif IN_KAGGLE:
    ROOT_DIR = Path('/kaggle/working')
else:
    # Local / VS Code
    ROOT_DIR = Path('.').resolve().parent
    if not (ROOT_DIR / 'scripts').exists():
        ROOT_DIR = Path('.').resolve()

# Add scripts to path
SCRIPTS_DIR = ROOT_DIR / 'scripts'
if str(SCRIPTS_DIR) not in sys.path:
    sys.path.insert(0, str(SCRIPTS_DIR))
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

DATA_DIR = ROOT_DIR / 'data'
OUTPUT_DIR = ROOT_DIR / 'output'
RESULTS_DIR = OUTPUT_DIR / 'results'
FIGURES_DIR = OUTPUT_DIR / 'figures'
MODELS_DIR = OUTPUT_DIR / 'models'

for d in [OUTPUT_DIR, RESULTS_DIR, FIGURES_DIR, MODELS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print(f"Root: {ROOT_DIR}")
print(f"Data: {DATA_DIR}")
print(f"Scripts: {SCRIPTS_DIR}")

In [ ]:
# =============================================================================
# CELL 2: Import from Project Scripts
# =============================================================================

# Standard imports
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    roc_auc_score, average_precision_score, roc_curve, precision_recall_curve,
    log_loss, brier_score_loss
)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import json
import pickle
from datetime import datetime
from collections import defaultdict

# Import from our scripts
from run_roberta import (
    set_seed, load_dataset, SUPPORTED_DATASETS,
    ToxicityDataset, build_model, train_one_epoch, evaluate,
    expected_calibration_error, fit_temperature, fit_isotonic
)
from run_tfidf_baselines import (
    evaluate_sklearn_classifier, train_and_evaluate_tfidf
)
from ood_algorithms import (
    MaxSoftmaxOOD, ODIN_OOD, EnergyOOD, MahalanobisOOD,
    CORAL, MMD,
    TemperatureScaling, IsotonicCalibration, PlattScaling
)
from fairness_metrics import compute_group_fairness

print("✓ Imported from project scripts")

In [ ]:
# =============================================================================
# CELL 3: Configuration
# =============================================================================

# Device setup
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.cuda.is_available():
    GPU_NAME = torch.cuda.get_device_name(0)
    GPU_MEM = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {GPU_NAME} ({GPU_MEM:.1f} GB)")
else:
    print("Running on CPU")

# Experiment config
CONFIG = {
    'seed': 42,
    'datasets': ['jigsaw', 'civil', 'hatexplain'],
    
    # RoBERTa training
    'model_name': 'roberta-base',
    'max_length': 128,
    'batch_size': 64 if 'A100' in globals().get('GPU_NAME', '') else 32,
    'epochs': 5,
    'learning_rate': 2e-5,
    'warmup_ratio': 0.1,
    
    # TF-IDF
    'tfidf_max_features': 50000,
    'tfidf_ngram_range': (1, 2),
    
    # Paths
    'data_dir': str(DATA_DIR),
    'save_models': True,
}

set_seed(CONFIG['seed'])
print(f"\nConfig: {json.dumps(CONFIG, indent=2, default=str)}")

In [ ]:
# =============================================================================
# CELL 4: Load All Datasets
# =============================================================================

data = {}

for name in CONFIG['datasets']:
    print(f"\nLoading {name}...")
    data[name] = {}
    
    for split in ['train', 'val', 'test']:
        try:
            df = load_dataset(name, split, CONFIG['data_dir'])
            data[name][split] = df
            pos_rate = df['label'].mean()
            print(f"  {split}: {len(df):,} samples, {pos_rate:.1%} positive")
        except FileNotFoundError as e:
            print(f"  {split}: Not found")

print("\n✓ Data loaded")

In [ ]:
# =============================================================================
# CELL 5: Results Tracker (Lightweight)
# =============================================================================

class Results:
    """Simple results storage."""
    def __init__(self):
        self.classification = []
        self.ood = []
        self.calibration = []
        self.fairness = []
        self.predictions = {}
    
    def add(self, category, **kwargs):
        getattr(self, category).append(kwargs)
    
    def get_df(self, category):
        return pd.DataFrame(getattr(self, category))
    
    def save(self, path):
        data = {
            'classification': self.classification,
            'ood': self.ood,
            'calibration': self.calibration,
            'fairness': self.fairness,
        }
        with open(path, 'w') as f:
            json.dump(data, f, indent=2, default=float)

results = Results()
print("✓ Results tracker initialized")

---
## Section 1: TF-IDF Baseline Experiments

Using `train_and_evaluate_tfidf` from `scripts/run_tfidf_baselines.py`

In [ ]:
# =============================================================================
# CELL 6: TF-IDF Baselines - Train & Cross-Domain Eval
# =============================================================================

print("="*60)
print("TF-IDF BASELINE EXPERIMENTS")
print("="*60)

tfidf_results = {}

for source in CONFIG['datasets']:
    if source not in data or 'train' not in data[source]:
        continue
    
    print(f"\n{'='*40}\nTraining on {source.upper()}\n{'='*40}")
    
    # Use existing function from run_tfidf_baselines.py
    for model_type in ['logreg', 'svm']:
        print(f"\n  TF-IDF + {model_type.upper()}:")
        
        # Vectorize
        vectorizer = TfidfVectorizer(
            max_features=CONFIG['tfidf_max_features'],
            ngram_range=CONFIG['tfidf_ngram_range'],
            min_df=5, sublinear_tf=True
        )
        X_train = vectorizer.fit_transform(data[source]['train']['text'].fillna(''))
        y_train = data[source]['train']['label'].values
        
        # Train
        if model_type == 'logreg':
            model = LogisticRegression(max_iter=1000, class_weight='balanced', n_jobs=-1)
        else:
            model = LinearSVC(max_iter=5000, dual=False, class_weight='balanced')
        model.fit(X_train, y_train)
        
        # Evaluate on all test sets
        for target in CONFIG['datasets']:
            if target not in data or 'test' not in data[target]:
                continue
            
            X_test = vectorizer.transform(data[target]['test']['text'].fillna(''))
            y_test = data[target]['test']['label'].values
            
            # Use existing evaluation function
            metrics = evaluate_sklearn_classifier(X_test, y_test, model, model_type)
            
            # Store
            eval_type = 'in-domain' if source == target else 'cross-domain'
            results.add('classification',
                model=f'TF-IDF-{model_type.upper()}', source=source, target=target,
                eval_type=eval_type, **metrics
            )
            
            marker = "★" if source == target else "→"
            print(f"    {marker} {target}: F1={metrics['f1']:.4f}, AUROC={metrics.get('auroc', 0):.4f}")
        
        # Save model
        if CONFIG['save_models']:
            key = f"tfidf_{model_type}_{source}"
            tfidf_results[key] = {'vectorizer': vectorizer, 'model': model}

print("\n✓ TF-IDF experiments complete")

---
## Section 2: RoBERTa Training & Cross-Domain Evaluation

Using `ToxicityDataset`, `build_model`, `train_one_epoch`, `evaluate` from `scripts/run_roberta.py`

In [ ]:
# =============================================================================
# CELL 7: RoBERTa Training
# =============================================================================

from transformers import AutoTokenizer, get_linear_schedule_with_warmup
from torch.optim import AdamW

print("="*60)
print("RoBERTa TRAINING & CROSS-DOMAIN EVALUATION")
print("="*60)

roberta_models = {}
training_histories = {}
eval_results = {}  # Store logits/features for OOD detection

tokenizer = AutoTokenizer.from_pretrained(CONFIG['model_name'])

for source in CONFIG['datasets']:
    if source not in data or 'train' not in data[source]:
        continue
    
    print(f"\n{'='*50}\nTraining RoBERTa on {source.upper()}\n{'='*50}")
    
    train_df = data[source]['train']
    val_df = data[source]['val']
    
    # Create datasets using imported class
    train_dataset = ToxicityDataset(
        train_df['text'].tolist(), train_df['label'].tolist(),
        tokenizer, CONFIG['max_length']
    )
    val_dataset = ToxicityDataset(
        val_df['text'].tolist(), val_df['label'].tolist(),
        tokenizer, CONFIG['max_length']
    )
    
    train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=2)
    
    # Build model using imported function
    model = build_model(CONFIG['model_name'], num_labels=2).to(DEVICE)
    
    # Optimizer & scheduler
    optimizer = AdamW(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=0.01)
    total_steps = len(train_loader) * CONFIG['epochs']
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=int(total_steps * CONFIG['warmup_ratio']),
        num_training_steps=total_steps
    )
    
    history = {'train_loss': [], 'val_loss': [], 'val_f1': []}
    best_f1, best_state = 0, None
    
    for epoch in range(CONFIG['epochs']):
        # Train using imported function (signature: model, dataloader, optimizer, scheduler, device)
        train_loss = train_one_epoch(model, train_loader, optimizer, scheduler, DEVICE)
        history['train_loss'].append(train_loss)
        
        # Validate using imported function (signature: model, dataloader, device, return_probs)
        val_metrics = evaluate(model, val_loader, DEVICE, return_probs=True)
        history['val_loss'].append(val_metrics.get('nll', 0))
        history['val_f1'].append(val_metrics['f1'])
        
        print(f"  Epoch {epoch+1}/{CONFIG['epochs']}: "
              f"Train Loss={train_loss:.4f}, Val F1={val_metrics['f1']:.4f}")
        
        if val_metrics['f1'] > best_f1:
            best_f1 = val_metrics['f1']
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
    
    # Load best model
    if best_state:
        model.load_state_dict({k: v.to(DEVICE) for k, v in best_state.items()})
    
    roberta_models[source] = model
    training_histories[source] = history
    print(f"  ✓ Best Val F1: {best_f1:.4f}")
    
    # Cross-domain evaluation
    print(f"\n  Cross-domain evaluation:")
    for target in CONFIG['datasets']:
        if target not in data or 'test' not in data[target]:
            continue
        
        test_df = data[target]['test']
        test_dataset = ToxicityDataset(
            test_df['text'].tolist(), test_df['label'].tolist(),
            tokenizer, CONFIG['max_length']
        )
        test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=2)
        
        # Evaluate and get logits (use return_probs=True to get logits and probs)
        metrics = evaluate(model, test_loader, DEVICE, return_probs=True)
        
        eval_key = f"{source}_to_{target}"
        eval_results[eval_key] = {
            'logits': metrics.get('logits'),
            'labels': metrics.get('labels'),
            'probs': metrics.get('pos_probs'),  # Use pos_probs (1D array for positive class)
        }
        
        eval_type = 'in-domain' if source == target else 'cross-domain'
        results.add('classification',
            model='RoBERTa', source=source, target=target,
            eval_type=eval_type, 
            accuracy=metrics['accuracy'], f1=metrics['f1'],
            auroc=metrics.get('auroc', 0), ece=metrics.get('ece', 0)
        )
        
        marker = "★" if source == target else "→"
        print(f"    {marker} {target}: F1={metrics['f1']:.4f}, AUROC={metrics.get('auroc', 0):.4f}")
    
    # Save model
    if CONFIG['save_models']:
        torch.save(model.state_dict(), MODELS_DIR / f'roberta_{source}.pt')
    
    # Clear cache
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

print("\n✓ RoBERTa training complete")

---
## Section 3: OOD Detection & Calibration

Using classes from `scripts/ood_algorithms.py`:
- OOD: `MaxSoftmaxOOD`, `ODIN_OOD`, `EnergyOOD`, `MahalanobisOOD`
- Domain Shift: `CORAL`, `MMD`
- Calibration: `TemperatureScaling`, `IsotonicCalibration`, `PlattScaling`

In [ ]:
# =============================================================================
# CELL 8: OOD Detection (Using Imported Classes)
# =============================================================================

print("="*60)
print("OOD DETECTION")
print("="*60)

# Initialize OOD detectors from ood_algorithms.py
ood_detectors = {
    'MaxSoftmax': MaxSoftmaxOOD(),
    'ODIN (T=1000)': ODIN_OOD(temperature=1000),
    'Energy': EnergyOOD(),
}

ood_results_dict = {}

for source in CONFIG['datasets']:
    id_key = f"{source}_to_{source}"
    if id_key not in eval_results or eval_results[id_key]['logits'] is None:
        continue
    
    id_logits = eval_results[id_key]['logits']
    id_labels = eval_results[id_key]['labels']
    
    print(f"\n{source.upper()} as In-Distribution:")
    
    for target in CONFIG['datasets']:
        if target == source:
            continue
        
        ood_key = f"{source}_to_{target}"
        if ood_key not in eval_results or eval_results[ood_key]['logits'] is None:
            continue
        
        ood_logits = eval_results[ood_key]['logits']
        
        # Binary labels: 1=ID, 0=OOD
        y_true = np.concatenate([np.ones(len(id_logits)), np.zeros(len(ood_logits))])
        all_logits = np.vstack([id_logits, ood_logits])
        
        print(f"  vs {target} (OOD):")
        for name, detector in ood_detectors.items():
            scores = -detector.compute_scores(all_logits)  # Negate: higher = more ID
            auroc = roc_auc_score(y_true, scores)
            
            results.add('ood', method=name, source=source, target=target, auroc=auroc)
            ood_results_dict[f"{source}_vs_{target}_{name}"] = auroc
            
            print(f"    {name}: AUROC={auroc:.4f}")

print("\n✓ OOD detection complete")

In [ ]:
# =============================================================================
# CELL 9: Calibration Analysis (Using Imported Classes)
# =============================================================================

print("="*60)
print("CALIBRATION ANALYSIS")
print("="*60)

calibration_results = {}

for source in roberta_models:
    print(f"\n{source.upper()} Model:")
    
    # Get validation data for fitting calibrators
    val_key = f"{source}_to_{source}"
    if val_key not in eval_results:
        continue
    
    val_logits = eval_results[val_key]['logits']
    val_labels = eval_results[val_key]['labels']
    val_probs = eval_results[val_key]['probs']  # This is pos_probs (1D)
    
    if val_logits is None:
        print(f"  Skipping {source} - no logits available")
        continue
    
    # Fit temperature scaling using imported class
    temp_scaler = TemperatureScaling()
    temp_scaler.fit(val_logits, val_labels)
    
    # For isotonic, we need 2D probs - create from pos_probs
    val_probs_2d = np.column_stack([1 - val_probs, val_probs])
    iso_calib = IsotonicCalibration()
    iso_calib.fit(val_probs_2d, val_labels)
    
    # Evaluate on all test sets
    for target in CONFIG['datasets']:
        test_key = f"{source}_to_{target}"
        if test_key not in eval_results:
            continue
        
        test_logits = eval_results[test_key]['logits']
        test_labels = eval_results[test_key]['labels']
        test_probs = eval_results[test_key]['probs']  # pos_probs (1D)
        
        if test_logits is None or test_probs is None:
            continue
        
        # Uncalibrated ECE (using imported function)
        ece_uncal, _ = expected_calibration_error(test_labels, test_probs)
        
        # Temperature Scaling
        cal_probs_temp = temp_scaler.calibrate(test_logits)
        ece_temp, _ = expected_calibration_error(test_labels, cal_probs_temp[:, 1])
        
        # Isotonic (need 2D input)
        test_probs_2d = np.column_stack([1 - test_probs, test_probs])
        cal_probs_iso = iso_calib.calibrate(test_probs_2d)
        ece_iso, _ = expected_calibration_error(test_labels, cal_probs_iso[:, 1])
        
        print(f"  {source}→{target}: ECE uncal={ece_uncal:.4f}, temp={ece_temp:.4f}, iso={ece_iso:.4f}")
        
        results.add('calibration', source=source, target=target, method='Uncalibrated', ece=ece_uncal)
        results.add('calibration', source=source, target=target, method='Temperature', ece=ece_temp)
        results.add('calibration', source=source, target=target, method='Isotonic', ece=ece_iso)
        
        calibration_results[test_key] = {
            'uncal': test_probs, 
            'temp': cal_probs_temp[:, 1], 
            'iso': cal_probs_iso[:, 1],
            'labels': test_labels
        }

print("\n✓ Calibration analysis complete")

---
## Section 4: Fairness Analysis

Using `compute_group_fairness` from `scripts/fairness_metrics.py`

In [ ]:
# =============================================================================
# CELL 10: Fairness Analysis (Using Imported Function)
# =============================================================================

print("="*60)
print("FAIRNESS ANALYSIS")
print("="*60)

# Identity keywords for text-based detection
IDENTITY_KEYWORDS = {
    'male': ['male', 'man', 'men', 'boy', 'he ', 'him', 'his ', 'father', 'son', 'brother'],
    'female': ['female', 'woman', 'women', 'girl', 'she ', 'her ', 'mother', 'daughter', 'sister'],
    'lgbtq': ['gay', 'lesbian', 'bisexual', 'transgender', 'queer', 'lgbt'],
    'christian': ['christian', 'christianity', 'church', 'catholic'],
    'muslim': ['muslim', 'islam', 'islamic', 'mosque'],
    'jewish': ['jewish', 'jew', 'judaism', 'synagogue'],
    'black': ['black', 'african american'],
    'white': ['white', 'caucasian'],
}

def detect_identity(text, keywords):
    """Check if text mentions identity group."""
    if pd.isna(text):
        return 0
    text_lower = str(text).lower()
    return 1 if any(kw in text_lower for kw in keywords) else 0

fairness_results = {}

for source in roberta_models:
    print(f"\n{source.upper()} Model:")
    
    for target in CONFIG['datasets']:
        test_key = f"{source}_to_{target}"
        if test_key not in eval_results:
            continue
        
        test_df = data[target]['test'].copy()
        test_labels = eval_results[test_key]['labels']
        test_probs = eval_results[test_key]['probs']
        test_preds = (test_probs >= 0.5).astype(int)
        
        # Add predictions to dataframe
        test_df = test_df.iloc[:len(test_labels)].copy()
        test_df['label'] = test_labels
        test_df['pred'] = test_preds
        test_df['pos_prob'] = test_probs
        
        # Detect identity groups
        group_cols = []
        for group_name, keywords in IDENTITY_KEYWORDS.items():
            col_name = f'g_{group_name}'
            test_df[col_name] = test_df['text'].apply(lambda x: detect_identity(x, keywords))
            if test_df[col_name].sum() > 50:  # Only include groups with enough samples
                group_cols.append(col_name)
        
        if not group_cols:
            print(f"  {source}→{target}: No identity groups detected")
            continue
        
        # Use imported function
        summary_df, per_group_df = compute_group_fairness(test_df, group_cols)
        
        fairness_results[test_key] = {'summary': summary_df, 'per_group': per_group_df}
        
        # Store results
        for _, row in summary_df.iterrows():
            group = row['group_col'].replace('g_', '')
            results.add('fairness', source=source, target=target, group=group,
                       dp_diff=row['dp_diff'], eop_diff=row['eop_diff'], eo_diff=row['eo_diff'])
        
        # Print summary
        avg_dp = summary_df['dp_diff'].mean()
        avg_eo = summary_df['eo_diff'].mean()
        print(f"  {source}→{target}: Avg DP Gap={avg_dp:.4f}, Avg EO Gap={avg_eo:.4f}")

print("\n✓ Fairness analysis complete")

---
## Section 5: Visualizations

All plots for the final report.

In [ ]:
# =============================================================================
# CELL 11: Plot Setup
# =============================================================================

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'figure.figsize': (12, 8), 'font.size': 11, 'figure.dpi': 150})

COLORS = {'jigsaw': '#1f77b4', 'civil': '#ff7f0e', 'hatexplain': '#2ca02c'}

def save_fig(fig, name):
    for fmt in ['png', 'pdf']:
        fig.savefig(FIGURES_DIR / f'{name}.{fmt}', dpi=300, bbox_inches='tight', facecolor='white')
    print(f"  ✓ Saved {name}")

print("✓ Plot settings configured")

In [ ]:
# =============================================================================
# CELL 12: Plot 1 - Classification Performance Heatmaps
# =============================================================================

print("Generating Classification Heatmaps...")

class_df = results.get_df('classification')

if len(class_df) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    for ax, model in zip(axes, ['RoBERTa', 'TF-IDF-LOGREG']):
        df = class_df[class_df['model'] == model]
        if len(df) == 0:
            continue
        
        # Create pivot table
        pivot = df.pivot_table(index='source', columns='target', values='f1', aggfunc='mean')
        
        im = ax.imshow(pivot.values, cmap='YlOrRd', vmin=0, vmax=1)
        ax.set_xticks(range(len(pivot.columns)))
        ax.set_yticks(range(len(pivot.index)))
        ax.set_xticklabels([c.capitalize() for c in pivot.columns])
        ax.set_yticklabels([c.capitalize() for c in pivot.index])
        ax.set_xlabel('Target')
        ax.set_ylabel('Source')
        ax.set_title(f'{model} F1 Score')
        
        # Annotate
        for i in range(len(pivot.index)):
            for j in range(len(pivot.columns)):
                val = pivot.values[i, j]
                if not np.isnan(val):
                    ax.text(j, i, f'{val:.3f}', ha='center', va='center', fontweight='bold', fontsize=11)
        
        plt.colorbar(im, ax=ax, shrink=0.8)
    
    plt.tight_layout()
    save_fig(fig, 'classification_heatmaps')
    plt.show()
else:
    print("  No classification results to plot")

In [ ]:
# =============================================================================
# CELL 13: Plot 2 - ROC & PR Curves
# =============================================================================

print("Generating ROC & PR Curves...")

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for idx, source in enumerate(CONFIG['datasets']):
    ax_roc = axes[0, idx]
    ax_pr = axes[1, idx]
    
    ax_roc.plot([0, 1], [0, 1], 'k--', alpha=0.5)
    
    for target in CONFIG['datasets']:
        key = f"{source}_to_{target}"
        if key not in eval_results or eval_results[key]['probs'] is None:
            continue
        
        probs = eval_results[key]['probs']
        labels = eval_results[key]['labels']
        
        # ROC
        fpr, tpr, _ = roc_curve(labels, probs)
        auc = roc_auc_score(labels, probs)
        style = '-' if source == target else '--'
        ax_roc.plot(fpr, tpr, style, label=f'{target} (AUC={auc:.3f})', color=COLORS[target], linewidth=2)
        
        # PR
        prec, rec, _ = precision_recall_curve(labels, probs)
        ap = average_precision_score(labels, probs)
        ax_pr.plot(rec, prec, style, label=f'{target} (AP={ap:.3f})', color=COLORS[target], linewidth=2)
    
    ax_roc.set_xlabel('FPR')
    ax_roc.set_ylabel('TPR')
    ax_roc.set_title(f'ROC: Trained on {source.capitalize()}')
    ax_roc.legend(loc='lower right', fontsize=9)
    
    ax_pr.set_xlabel('Recall')
    ax_pr.set_ylabel('Precision')
    ax_pr.set_title(f'PR: Trained on {source.capitalize()}')
    ax_pr.legend(loc='lower left', fontsize=9)

plt.tight_layout()
save_fig(fig, 'roc_pr_curves')
plt.show()

In [ ]:
# =============================================================================
# CELL 14: Plot 3 - Calibration Reliability Diagrams
# =============================================================================

from sklearn.calibration import calibration_curve

print("Generating Calibration Plots...")

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, source in enumerate(CONFIG['datasets']):
    ax = axes[idx]
    ax.plot([0, 1], [0, 1], 'k--', label='Perfect', alpha=0.5)
    
    for target in CONFIG['datasets']:
        key = f"{source}_to_{target}"
        if key not in calibration_results:
            continue
        
        probs = calibration_results[key]['uncal']
        labels = calibration_results[key]['labels']
        
        if probs is None or len(probs) == 0:
            continue
        
        prob_true, prob_pred = calibration_curve(labels, probs, n_bins=10, strategy='uniform')
        style = '-o' if source == target else '--s'
        ax.plot(prob_pred, prob_true, style, label=f'{target}', color=COLORS[target], markersize=5)
    
    ax.set_xlabel('Mean Predicted Probability')
    ax.set_ylabel('Fraction of Positives')
    ax.set_title(f'Calibration: {source.capitalize()} Model')
    ax.legend(loc='lower right', fontsize=9)
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])

plt.tight_layout()
save_fig(fig, 'calibration_reliability')
plt.show()

In [ ]:
# =============================================================================
# CELL 15: Plot 4 - OOD Detection & Fairness
# =============================================================================

print("Generating OOD Detection & Fairness Plots...")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# OOD Detection
ood_df = results.get_df('ood')
if len(ood_df) > 0:
    ax = axes[0]
    pivot = ood_df.pivot_table(index=['source', 'target'], columns='method', values='auroc')
    pivot.plot(kind='bar', ax=ax, width=0.8)
    ax.set_xlabel('Source vs Target')
    ax.set_ylabel('AUROC')
    ax.set_title('OOD Detection Performance')
    ax.axhline(y=0.5, color='red', linestyle='--', alpha=0.5, label='Random')
    ax.legend(loc='upper right', fontsize=9)
    ax.set_xticklabels([f"{s}→{t}" for s, t in pivot.index], rotation=45, ha='right')
else:
    axes[0].text(0.5, 0.5, 'No OOD results', ha='center', va='center')
    axes[0].set_title('OOD Detection')

# Fairness
fair_df = results.get_df('fairness')
if len(fair_df) > 0:
    ax = axes[1]
    # Aggregate by source-target pair
    agg = fair_df.groupby(['source', 'target'])[['dp_diff', 'eo_diff']].mean().reset_index()
    agg['pair'] = agg['source'] + '→' + agg['target']
    
    x = np.arange(len(agg))
    width = 0.35
    ax.bar(x - width/2, agg['dp_diff'], width, label='DP Gap', color='steelblue')
    ax.bar(x + width/2, agg['eo_diff'], width, label='EO Gap', color='coral')
    ax.set_xticks(x)
    ax.set_xticklabels(agg['pair'], rotation=45, ha='right')
    ax.set_ylabel('Gap Value')
    ax.set_title('Fairness Gaps (Lower is Better)')
    ax.axhline(y=0.1, color='green', linestyle='--', alpha=0.5)
    ax.legend()
else:
    axes[1].text(0.5, 0.5, 'No fairness results', ha='center', va='center')
    axes[1].set_title('Fairness Gaps')

plt.tight_layout()
save_fig(fig, 'ood_fairness')
plt.show()

In [ ]:
# =============================================================================
# CELL 16: Plot 5 - Training Curves
# =============================================================================

print("Generating Training Curves...")

if training_histories:
    n = len(training_histories)
    fig, axes = plt.subplots(1, n, figsize=(5*n, 4))
    if n == 1:
        axes = [axes]
    
    for ax, (source, hist) in zip(axes, training_histories.items()):
        epochs = range(1, len(hist['train_loss']) + 1)
        
        ax.plot(epochs, hist['train_loss'], 'b-', label='Train Loss', linewidth=2)
        ax.plot(epochs, hist['val_loss'], 'r-', label='Val Loss', linewidth=2)
        
        ax2 = ax.twinx()
        ax2.plot(epochs, hist['val_f1'], 'g--', label='Val F1', linewidth=2)
        ax2.set_ylabel('F1 Score', color='green')
        ax2.tick_params(axis='y', labelcolor='green')
        
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Loss')
        ax.set_title(f'Training: {source.capitalize()}')
        ax.legend(loc='upper left')
        ax2.legend(loc='upper right')
    
    plt.tight_layout()
    save_fig(fig, 'training_curves')
    plt.show()
else:
    print("  No training histories to plot")

---
## Section 6: Results Summary & Export

In [ ]:
# =============================================================================
# CELL 17: Results Summary
# =============================================================================

print("="*60)
print("RESULTS SUMMARY")
print("="*60)

# Classification
print("\n📊 CLASSIFICATION PERFORMANCE")
class_df = results.get_df('classification')
if len(class_df) > 0:
    roberta = class_df[class_df['model'] == 'RoBERTa']
    if len(roberta) > 0:
        print("\nRoBERTa Cross-Domain F1:")
        display(roberta.pivot_table(index='source', columns='target', values='f1').round(4))
        
        cross = roberta[roberta['source'] != roberta['target']]
        if len(cross) > 0:
            best = cross.loc[cross['f1'].idxmax()]
            print(f"\n✓ Best Cross-Domain: {best['source']}→{best['target']}: F1={best['f1']:.4f}")

# OOD Detection
print("\n\n🔍 OOD DETECTION")
ood_df = results.get_df('ood')
if len(ood_df) > 0:
    display(ood_df.pivot_table(index=['source', 'target'], columns='method', values='auroc').round(4))

# Calibration
print("\n\n📏 CALIBRATION (ECE)")
cal_df = results.get_df('calibration')
if len(cal_df) > 0:
    display(cal_df.pivot_table(index=['source', 'target'], columns='method', values='ece').round(4))

# Fairness
print("\n\n⚖️ FAIRNESS")
fair_df = results.get_df('fairness')
if len(fair_df) > 0:
    agg = fair_df.groupby(['source', 'target'])[['dp_diff', 'eo_diff']].mean()
    display(agg.round(4))

In [ ]:
# =============================================================================
# CELL 18: Export Results
# =============================================================================

print("="*60)
print("SAVING RESULTS")
print("="*60)

# Save JSON
results.save(RESULTS_DIR / 'all_results.json')
print(f"✓ JSON: {RESULTS_DIR / 'all_results.json'}")

# Save CSVs
for name in ['classification', 'ood', 'calibration', 'fairness']:
    df = results.get_df(name)
    if len(df) > 0:
        df.to_csv(RESULTS_DIR / f'{name}_results.csv', index=False)
        print(f"✓ CSV: {name}_results.csv ({len(df)} rows)")

# Save training histories
if training_histories:
    with open(RESULTS_DIR / 'training_histories.json', 'w') as f:
        json.dump({k: {kk: [float(x) for x in vv] for kk, vv in v.items()} 
                   for k, v in training_histories.items()}, f, indent=2)
    print(f"✓ Training histories saved")

# List figures
figs = list(FIGURES_DIR.glob('*.png'))
print(f"\n✓ {len(figs)} figures saved to {FIGURES_DIR}")

print("\n" + "="*60)
print("🎉 ALL EXPERIMENTS COMPLETE!")
print("="*60)